## Loading Libraries and preprocessing data

In [2]:
!unzip -a '/content/drive/My Drive/NLP/input/fake-news.zip'

Archive:  /content/drive/My Drive/NLP/input/fake-news.zip
  inflating: submit.csv              [binary]
  inflating: test.csv                [binary]
  inflating: train.csv               [binary]


In [4]:
!pip install texthero

     |████████████████████████████████| 245kB 14.0MB/s 
     |████████████████████████████████| 1.4MB 38.4MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp36-none-any.whl size=1434674 sha256=84cba267fc0bd6ffe92c80128c9afec9c3ef3e128d2d99faa7565a6469545037
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import gensim.downloader as api
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import nltk
import warnings
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential, layers, losses, callbacks, metrics, Model
from tensorflow.keras.layers import Input, LSTM, Flatten, BatchNormalization, Dense, Activation, Concatenate
import tensorflow_hub as hub
import tensorflow.keras.backend as K
import texthero as hero
from tqdm.notebook import tqdm
import operator 
warnings.filterwarnings('ignore')
tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
df = pd.read_csv('train.csv')
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


### Cleaning text

In [5]:
?BatchNormalization

In [3]:
?hero.clean

In [4]:
df['text'] = hero.clean(df['text'])
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide even see comey letter jason cha...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,ever get feeling life circles roundabout rathe...,0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Tokenization

In [ ]:
text = hero.tokenize(df['text'])
text[:10]

0    [house, dem, aide, even, see, comey, letter, j...
1    [ever, get, feeling, life, circles, roundabout...
2    [truth, might, get, fired, october, tension, i...
3    [videos, civilians, killed, single, us, airstr...
4    [print, iranian, woman, sentenced, six, years,...
5    [trying, times, jackie, mason, voice, reason, ...
6    [ever, wonder, britain, iconic, pop, pianist, ...
7    [paris, france, chose, idealistic, traditional...
8    [donald, j, trump, scheduled, make, highly, an...
9    [week, michael, flynn, resigned, national, sec...
Name: text, dtype: object

### Embeddings

In [ ]:
api.info()['models'].keys()

dict_keys(['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis'])

In [ ]:
glove_model = api.load('glove-wiki-gigaword-100')

[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [ ]:
def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [ ]:
def create_embeddings(model, data):

    embedded_text = np.zeros((len(data), model.vector_size))
    
    for index, sent in tqdm(enumerate(data), total=len(data)):
        
        sent_vec = np.zeros(100)
        
        for word in sent:
            
            if word in model.wv:
                
                word_vec = model.wv[word]
                sent_vec += word_vec
                
        embedded_text[index] = sent_vec
        
    return embedded_text

In [ ]:
vocab = build_vocab(text)
oov = check_coverage(vocab, glove_model)


Found embeddings for 57.40% of vocab
Found embeddings for  97.93% of all text


In [ ]:
e_text = create_embeddings(glove_model, text)

In [ ]:
e_text_train = np.array(e_text[:17000]).reshape(17000, 100, 1)
e_text_valid = np.array(e_text[17000:]).reshape(3800, 100, 1)

In [ ]:
label = df['label']

## SimpleRNN Model

In [ ]:
rnn_model = Sequential([
    layers.SimpleRNN(128, input_shape=(100,1)),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
rnn_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 32s 60ms/step - loss: 0.6202 - accuracy: 0.6466 - val_loss: 0.6054 - val_accuracy: 0.6468
Epoch 2/3
532/532 [==============================] - 32s 59ms/step - loss: 0.5902 - accuracy: 0.6682 - val_loss: 0.5813 - val_accuracy: 0.6689
Epoch 3/3
532/532 [==============================] - 31s 59ms/step - loss: 0.5702 - accuracy: 0.6939 - val_loss: 0.5594 - val_accuracy: 0.7053


### LSTM Model

In [ ]:
lstm_model = Sequential([
    layers.LSTM(128, input_shape = (100,1)),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
lstm_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 4s 8ms/step - loss: 0.5714 - accuracy: 0.6912 - val_loss: 0.5463 - val_accuracy: 0.7137
Epoch 2/3
532/532 [==============================] - 4s 8ms/step - loss: 0.5181 - accuracy: 0.7416 - val_loss: 0.4904 - val_accuracy: 0.7587
Epoch 3/3
532/532 [==============================] - 4s 7ms/step - loss: 0.4832 - accuracy: 0.7700 - val_loss: 0.5079 - val_accuracy: 0.7487


## GRU model

In [ ]:
gru_model = Sequential([
    layers.GRU(128, input_shape = (100,1)),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
gru_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 4s 8ms/step - loss: 0.5896 - accuracy: 0.6767 - val_loss: 0.5611 - val_accuracy: 0.7008
Epoch 2/3
532/532 [==============================] - 4s 7ms/step - loss: 0.5489 - accuracy: 0.7106 - val_loss: 0.5173 - val_accuracy: 0.7368
Epoch 3/3
532/532 [==============================] - 4s 7ms/step - loss: 0.5070 - accuracy: 0.7456 - val_loss: 0.4960 - val_accuracy: 0.7537


## Bidirectional LSTM

In [ ]:
bilstm_model = Sequential([
    layers.Input(shape = (100,1)),
    layers.Bidirectional(layers.LSTM(128)),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
bilstm_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 7s 14ms/step - loss: 0.5296 - accuracy: 0.7292 - val_loss: 0.4808 - val_accuracy: 0.7632
Epoch 2/3
532/532 [==============================] - 7s 13ms/step - loss: 0.4722 - accuracy: 0.7718 - val_loss: 0.4722 - val_accuracy: 0.7729
Epoch 3/3
532/532 [==============================] - 7s 13ms/step - loss: 0.4402 - accuracy: 0.7938 - val_loss: 0.4602 - val_accuracy: 0.7847


## Bidirectional GRU

In [ ]:
bigru_model = Sequential([
    layers.Input(shape = (100,1)),
    layers.Bidirectional(layers.GRU(128)),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
bigru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
bigru_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 7s 12ms/step - loss: 0.5421 - accuracy: 0.7227 - val_loss: 0.5019 - val_accuracy: 0.7442
Epoch 2/3
532/532 [==============================] - 6s 11ms/step - loss: 0.4829 - accuracy: 0.7670 - val_loss: 0.4721 - val_accuracy: 0.7705
Epoch 3/3
532/532 [==============================] - 6s 11ms/step - loss: 0.4429 - accuracy: 0.7907 - val_loss: 0.4380 - val_accuracy: 0.7868


## CNN along with LSTM

In [ ]:
combined_model = Sequential([
    layers.Input(shape=(100,1)),
    layers.LSTM(128, return_sequences=True),
    layers.Conv1D(64, 3),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
combined_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 4s 8ms/step - loss: 0.4218 - accuracy: 0.8043 - val_loss: 0.3609 - val_accuracy: 0.8363
Epoch 2/3
532/532 [==============================] - 4s 8ms/step - loss: 0.3577 - accuracy: 0.8408 - val_loss: 0.3459 - val_accuracy: 0.8516
Epoch 3/3
532/532 [==============================] - 4s 8ms/step - loss: 0.3400 - accuracy: 0.8492 - val_loss: 0.3448 - val_accuracy: 0.8495


## CNN with GRU

In [ ]:
combined_model = Sequential([
    layers.Input(shape=(100,1)),
    layers.GRU(128, return_sequences=True),
    layers.Conv1D(64, 3),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
combined_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 4s 8ms/step - loss: 0.4407 - accuracy: 0.7976 - val_loss: 0.3723 - val_accuracy: 0.8305
Epoch 2/3
532/532 [==============================] - 4s 7ms/step - loss: 0.3691 - accuracy: 0.8374 - val_loss: 0.3575 - val_accuracy: 0.8384
Epoch 3/3
532/532 [==============================] - 4s 7ms/step - loss: 0.3521 - accuracy: 0.8468 - val_loss: 0.3804 - val_accuracy: 0.8361


## CNN with Bidirectional LSTM

In [ ]:
combined_model = Sequential([
    layers.Input(shape=(100,1)),
    layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    layers.Conv1D(64, 3),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
combined_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 8s 14ms/step - loss: 0.4153 - accuracy: 0.8101 - val_loss: 0.3719 - val_accuracy: 0.8405
Epoch 2/3
532/532 [==============================] - 7s 13ms/step - loss: 0.3625 - accuracy: 0.8405 - val_loss: 0.3497 - val_accuracy: 0.8429
Epoch 3/3
532/532 [==============================] - 7s 13ms/step - loss: 0.3397 - accuracy: 0.8505 - val_loss: 0.3510 - val_accuracy: 0.8482


## CNN with Bidirectional  GRU

In [ ]:
combined_model = Sequential([
    layers.Input(shape=(100,1)),
    layers.Bidirectional(layers.GRU(128, return_sequences=True)),
    layers.Conv1D(64, 3),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
combined_model.fit(e_text_train, label[:17000], epochs=3, validation_data=(e_text_valid, label[17000:]))

Epoch 1/3
532/532 [==============================] - 7s 13ms/step - loss: 0.4563 - accuracy: 0.7949 - val_loss: 0.5577 - val_accuracy: 0.7553
Epoch 2/3
532/532 [==============================] - 6s 12ms/step - loss: 0.3772 - accuracy: 0.8314 - val_loss: 0.3564 - val_accuracy: 0.8482
Epoch 3/3
532/532 [==============================] - 6s 12ms/step - loss: 0.3625 - accuracy: 0.8396 - val_loss: 0.3726 - val_accuracy: 0.8350


## Using custom embeddings

In [ ]:
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,house dem aide even see comey letter jason cha...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,ever get feeling life circles roundabout rathe...,0


In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(oov_token='<OOV>', num_words = 10000)
tokenizer.fit_on_texts(text)
sequence = tokenizer.texts_to_sequences(text)
sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=2000)

In [ ]:
vocab_size = len(tokenizer.word_index.keys())
max_len = len(sequence[0])

In [ ]:
train_seq = sequence[:17000]
valid_seq = sequence[17000:]

### RNN model

In [ ]:
rnn_model = Sequential([
    layers.Embedding(vocab_size, 20),
    layers.SimpleRNN(128),
    layers.Dense(1, activation='sigmoid')
])

In [ ]:
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
rnn_model.fit(train_seq, label[:17000], epochs=3, validation_data=(valid_seq, label[17000:]))

### LSTM model

In [ ]:
lstm_model = Sequential([
    layers.Embedding(vocab_size, 20),
    layers.LSTM(128),
    layers.Dense(1, activation='sigmoid')
])

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

lstm_model.fit(train_seq, label[:17000], epochs=3, validation_data=(valid_seq, label[17000:]))

Epoch 1/3
532/532 [==============================] - 67s 126ms/step - loss: 0.2923 - accuracy: 0.8804 - val_loss: 0.1885 - val_accuracy: 0.9353
Epoch 2/3
532/532 [==============================] - 67s 125ms/step - loss: 0.1436 - accuracy: 0.9487 - val_loss: 0.1773 - val_accuracy: 0.9384
Epoch 3/3
532/532 [==============================] - 66s 125ms/step - loss: 0.2831 - accuracy: 0.8896 - val_loss: 0.2710 - val_accuracy: 0.9032


### Bidirectional LSTM

In [ ]:
bilstm_model = Sequential([
    layers.Embedding(vocab_size, 20),
    layers.Bidirectional(LSTM(128)),
    layers.Dense(1, activation='sigmoid')
])

bilstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

bilstm_model.fit(train_seq, label[:17000], epochs=3, validation_data=(valid_seq, label[17000:]))

Epoch 1/3
532/532 [==============================] - 111s 209ms/step - loss: 0.3574 - accuracy: 0.8549 - val_loss: 1.3114 - val_accuracy: 0.6392
Epoch 2/3
532/532 [==============================] - 113s 213ms/step - loss: 0.1736 - accuracy: 0.9395 - val_loss: 0.2224 - val_accuracy: 0.9150
Epoch 3/3
532/532 [==============================] - 114s 214ms/step - loss: 0.1272 - accuracy: 0.9559 - val_loss: 0.2031 - val_accuracy: 0.9266


### CNN with LSTM

In [ ]:
combined_model = Sequential([
    layers.Embedding(vocab_size, 20),
    layers.Conv1D(64, 3),
    layers.LSTM(128),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

combined_model.fit(train_seq, label[:17000], epochs=3, validation_data=(valid_seq, label[17000:]))

Epoch 1/3
532/532 [==============================] - 68s 127ms/step - loss: 0.3375 - accuracy: 0.8582 - val_loss: 0.2896 - val_accuracy: 0.8926
Epoch 2/3
532/532 [==============================] - 67s 126ms/step - loss: 0.1698 - accuracy: 0.9351 - val_loss: 0.1868 - val_accuracy: 0.9221
Epoch 3/3
532/532 [==============================] - 67s 126ms/step - loss: 0.1221 - accuracy: 0.9533 - val_loss: 0.1738 - val_accuracy: 0.9347


### CNN with Bidirectional LSTM

In [ ]:
combined_model = Sequential([
    layers.Embedding(vocab_size, 20),
    layers.Conv1D(64, 3),
    layers.Bidirectional(LSTM(128)),
    layers.Flatten(),
    layers.Dense(1, activation='sigmoid')
])

combined_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

combined_model.fit(train_seq, label[:17000], epochs=3, validation_data=(valid_seq, label[17000:]))

Epoch 1/3
532/532 [==============================] - 112s 211ms/step - loss: 0.3509 - accuracy: 0.8479 - val_loss: 0.2208 - val_accuracy: 0.9203
Epoch 2/3
532/532 [==============================] - 112s 210ms/step - loss: 0.1485 - accuracy: 0.9484 - val_loss: 0.2862 - val_accuracy: 0.8718
Epoch 3/3
532/532 [==============================] - 112s 210ms/step - loss: 0.0881 - accuracy: 0.9699 - val_loss: 0.2105 - val_accuracy: 0.9403


## Using Encoder Based Embeddings

In [ ]:
model = "https://tfhub.dev/google/universal-sentence-encoder/4"

In [ ]:
hub_layer = hub.KerasLayer(model, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(text[:3])

<tf.Tensor: shape=(3, 512), dtype=float32, numpy=
array([[ 0.04574309, -0.0457449 ,  0.04199997, ..., -0.04574504,
        -0.04574428,  0.04574506],
       [ 0.04512052, -0.04613996, -0.04611556, ..., -0.04613769,
        -0.04613996,  0.04613031],
       [ 0.04513958, -0.04513403,  0.04513819, ..., -0.0451396 ,
        -0.0451396 ,  0.0451396 ]], dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_64"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_20 (KerasLayer)  (None, 512)               256797824 
_________________________________________________________________
dense_68 (Dense)             (None, 16)                8208      
_________________________________________________________________
dense_69 (Dense)             (None, 1)                 17        
Total params: 256,806,049
Trainable params: 256,806,049
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
x_val = text[:17000]
partial_x_train = text[17000:]

y_val = label[:17000]
partial_y_train = label[17000:]

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=3,
                    validation_data=(x_val, y_val))

Epoch 1/3
119/119 [==============================] - 51s 425ms/step - loss: 0.4362 - accuracy: 0.7984 - val_loss: 0.2532 - val_accuracy: 0.9051
Epoch 2/3
119/119 [==============================] - 50s 423ms/step - loss: 0.0787 - accuracy: 0.9800 - val_loss: 0.2598 - val_accuracy: 0.9076
Epoch 3/3
119/119 [==============================] - 50s 422ms/step - loss: 0.0097 - accuracy: 0.9992 - val_loss: 0.2858 - val_accuracy: 0.9225


In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Reshape((512, 1)))
model.add(tf.keras.layers.LSTM(16))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_20 (KerasLayer)  (None, 512)               256797824 
_________________________________________________________________
reshape_3 (Reshape)          (None, 512, 1)            0         
_________________________________________________________________
lstm_54 (LSTM)               (None, 16)                1152      
_________________________________________________________________
dense_70 (Dense)             (None, 1)                 17        
Total params: 256,798,993
Trainable params: 256,798,993
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0, name='accuracy')])

In [ ]:
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=3,
                    validation_data=(x_val, y_val))

Epoch 1/3
119/119 [==============================] - 55s 465ms/step - loss: 0.5622 - accuracy: 0.6866 - val_loss: 0.3683 - val_accuracy: 0.9201
Epoch 2/3
119/119 [==============================] - 55s 460ms/step - loss: 0.2031 - accuracy: 0.9871 - val_loss: 0.2957 - val_accuracy: 0.9268
Epoch 3/3
119/119 [==============================] - 55s 465ms/step - loss: 0.1162 - accuracy: 0.9916 - val_loss: 0.3360 - val_accuracy: 0.9014


**Using tf-hub gave great results and that too without any cleaning of the text. Although it did show overfitting but that could be handled.**